In [1]:
import numpy as np 
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt

path = "./"  #Insert path here
database = path + 'database.sqlite'

In [2]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [3]:
explore = pd.read_sql("""SELECT *
                         FROM Match
                         ORDER BY date
                         LIMIT 5;""", conn)

explore

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,24559,24558,24558,2008/2009,1,2008-07-18 00:00:00,486263,10192,9931,1,...,None,None,None,None,None,None,None,None,None,None
1,24560,24558,24558,2008/2009,1,2008-07-19 00:00:00,486264,9930,10179,3,...,None,None,None,None,None,None,None,None,None,None
2,24561,24558,24558,2008/2009,1,2008-07-20 00:00:00,486265,10199,9824,1,...,None,None,None,None,None,None,None,None,None,None
3,24562,24558,24558,2008/2009,1,2008-07-20 00:00:00,486266,7955,10243,1,...,None,None,None,None,None,None,None,None,None,None
4,24613,24558,24558,2008/2009,2,2008-07-23 00:00:00,486267,9931,9956,1,...,None,None,None,None,None,None,None,None,None,None


In [57]:
#Which league has the highest home win percentage rate?

most_home_league_wins = pd.read_sql("""SELECT l.name, 
                                         ROUND(AVG(CASE WHEN m.home_team_goal > away_team_goal THEN 1
                                                ELSE 0 END), 2) AS home_win_prct
                                  FROM Match as m
                                  LEFT JOIN League as l
                                  USING (country_id)
                                  GROUP BY l.name
                                  ORDER BY home_win_prct DESC
                                  LIMIT 50;""",conn)

most_home_league_wins

,name,home_win_prct
0,Spain LIGA BBVA,0.49
1,Netherlands Eredivisie,0.48
2,Italy Serie A,0.47
3,Belgium Jupiler League,0.47
4,Switzerland Super League,0.46
5,England Premier League,0.46
6,Poland Ekstraklasa,0.45
7,Germany 1. Bundesliga,0.45
8,France Ligue 1,0.45
9,Portugal Liga ZON Sagres,0.44


In [5]:
#My favorite team is Barcelona. Their biggest rival is Real Madrid. 
#How many times have these two teams faced each other and who has the most wins?

goals = pd.read_sql("""SELECT team_api_id, team_long_name
                       FROM Team
                       WHERE team_long_name LIKE '%Barcelo%';""", conn)

goals




,team_api_id,team_long_name
0,8634,FC Barcelona


In [6]:
countries = pd.read_sql("""SELECT *
                        FROM Country;""", conn)
countries

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [7]:
leagues = pd.read_sql("""SELECT *
                        FROM League
                        JOIN Country ON Country.id = League.country_id;""", conn)
leagues

,id,country_id,name,id,name
0,1,1,Belgium Jupiler League,1,Belgium
1,1729,1729,England Premier League,1729,England
2,4769,4769,France Ligue 1,4769,France
3,7809,7809,Germany 1. Bundesliga,7809,Germany
4,10257,10257,Italy Serie A,10257,Italy
5,13274,13274,Netherlands Eredivisie,13274,Netherlands
6,15722,15722,Poland Ekstraklasa,15722,Poland
7,17642,17642,Portugal Liga ZON Sagres,17642,Portugal
8,19694,19694,Scotland Premier League,19694,Scotland
9,21518,21518,Spain LIGA BBVA,21518,Spain


In [6]:
teams = pd.read_sql("""SELECT *
                        FROM Team
                        ORDER BY team_long_name 
                        LIMIT 10;""", conn)
teams

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,16848,8350,29,1. FC Kaiserslautern,KAI
1,15624,8722,31,1. FC Köln,FCK
2,16239,8165,171,1. FC Nürnberg,NUR
3,16243,9905,169,1. FSV Mainz 05,MAI
4,11817,8576,614,AC Ajaccio,AJA
5,11074,108893,111989,AC Arles-Avignon,ARL
6,49116,6493,1714,AC Bellinzona,BEL
7,26560,10217,650,ADO Den Haag,HAA
8,9537,8583,57,AJ Auxerre,AUX
9,9547,9829,69,AS Monaco,MON


In [8]:
players = pd.read_sql("""SELECT *
                         FROM Player;""", conn)

players

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [9]:
messi = pd.read_sql("""SELECT *
                         FROM Player
                         WHERE player_name='Lionel Messi';""", conn)

messi

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,6176,30981,Lionel Messi,158023,1987-06-24 00:00:00,170.18,159


In [ ]:
#Highest scoring teams of each league
#Which are the teams that these highest scoring teams love to score on (use group by)



In [10]:
#Matches that have the biggest goal difference 

goals = pd.read_sql("""SELECT season, league.name AS league_name, team.team_long_name AS home_team, match.home_team_goal, team2.team_long_name AS away_team, match.away_team_goal
                         FROM Match AS match
                         INNER JOIN Team AS team ON match.home_team_api_id = team.team_api_id 
                         INNER JOIN Team AS team2 ON match.away_team_api_id = team2.team_api_id
                         INNER JOIN league AS league ON match.country_id = league.country_id
                         WHERE (match.home_team_goal - match.away_team_goal) > 7
                         GROUP BY team.team_long_name, team2.team_long_name
                         ORDER BY match.home_team_goal DESC
                         LIMIT 50;""", conn)

goals

,season,league_name,home_team,home_team_goal,away_team,away_team_goal
0,2010/2011,Netherlands Eredivisie,PSV,10,Feyenoord,0
1,2015/2016,Spain LIGA BBVA,Real Madrid CF,10,Rayo Vallecano,2
2,2010/2011,Scotland Premier League,Celtic,9,Aberdeen,0
3,2014/2015,Spain LIGA BBVA,Real Madrid CF,9,Granada CF,1
4,2009/2010,England Premier League,Tottenham Hotspur,9,Wigan Athletic,1
5,2012/2013,England Premier League,Chelsea,8,Aston Villa,0
6,2009/2010,England Premier League,Chelsea,8,Wigan Athletic,0
7,2011/2012,Spain LIGA BBVA,FC Barcelona,8,CA Osasuna,0
8,2014/2015,Germany 1. Bundesliga,FC Bayern Munich,8,Hamburger SV,0
9,2014/2015,England Premier League,Southampton,8,Sunderland,0


In [11]:
#We have coaches and players in various leagues complaining about the number of matches, is it true?
#Highest number of matches

burnout = pd.read_sql("""SELECT l.name AS league, 
                              COUNT(m.country_id) AS matches
                        FROM League as l
                        LEFT JOIN match as m
                        ON l.country_id = m.country_id
                        GROUP BY l.name
                        ORDER BY matches DESC;""", conn)

burnout

#It is more probable that a team from England Premier League, France Ligue 1, Spanish LIGA BBVA, and Italy Serie A to burnout from playing too many matches


,league,matches
0,England Premier League,3040
1,France Ligue 1,3040
2,Spain LIGA BBVA,3040
3,Italy Serie A,3017
4,Germany 1. Bundesliga,2448
5,Netherlands Eredivisie,2448
6,Portugal Liga ZON Sagres,2052
7,Poland Ekstraklasa,1920
8,Scotland Premier League,1824
9,Belgium Jupiler League,1728


In [30]:
#These are the professional teams in Spain's top league called "La Liga"

laliga = pd.read_sql("""SELECT DISTINCT m.home_team_api_id, t.team_long_name
                       FROM Match as m
                       LEFT JOIN Team as t
                       ON m.home_team_api_id = t.team_api_id
                       WHERE country_id = 21518
                       ORDER BY t.team_long_name;""", conn)

laliga

,home_team_api_id,team_long_name
0,8315,Athletic Club de Bilbao
1,9906,Atlético Madrid
2,8371,CA Osasuna
3,8388,CD Numancia
4,9867,CD Tenerife
5,7869,Córdoba CF
6,10268,Elche CF
7,8634,FC Barcelona
8,8305,Getafe CF
9,7878,Granada CF


In [58]:
#Which league has the highest home win percentage rate?

most_home_league_wins = pd.read_sql("""SELECT l.name, 
                                         ROUND(AVG(CASE WHEN m.home_team_goal > away_team_goal THEN 1
                                                ELSE 0 END), 2) AS home_win_prct
                                  FROM Match as m
                                  LEFT JOIN League as l
                                  USING (country_id)
                                  GROUP BY l.name
                                  ORDER BY home_win_prct DESC
                                  LIMIT 50;""",conn)

most_home_league_wins

,name,home_win_prct
0,Spain LIGA BBVA,0.49
1,Netherlands Eredivisie,0.48
2,Italy Serie A,0.47
3,Belgium Jupiler League,0.47
4,Switzerland Super League,0.46
5,England Premier League,0.46
6,Poland Ekstraklasa,0.45
7,Germany 1. Bundesliga,0.45
8,France Ligue 1,0.45
9,Portugal Liga ZON Sagres,0.44
